<a href="https://colab.research.google.com/github/Oreksandero/python_mate/blob/master/Python_Tableu_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Встановлення останньої версії google-cloud-bigquery
!pip install --upgrade google-cloud-bigquery

# Імпорт бібліотек
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
from google.colab import files

In [ ]:
# Аутентифікація в Google Colab для доступу до BigQuery
auth.authenticate_user()

# Створення клієнта BigQuery для проєкту "data-analytics-mate"
client = bigquery.Client(project="data-analytics-mate")

In [ ]:
# SQL запит для створення датасету
query = """
with session_info as (
 SELECT
    s.date,
    s.ga_session_id,
    sp.country,
    sp.continent,
    sp.channel,
    sp.device,
    ab.test,
    ab.test_group
 from `DA.ab_test` ab
 join `DA.session` s
 on ab.ga_session_id = s.ga_session_id
 join `DA.session_params` sp
 on sp.ga_session_id = ab.ga_session_id
),

session_with_orders as (
SELECT
    session_info.date,
    session_info.country,
    session_info.continent,
    session_info.channel,
    session_info.device,
    session_info.test,
    session_info.test_group,
  count(distinct o.ga_session_id) as session_with_orders
from `DA.order` o
join session_info
on o.ga_session_id = session_info.ga_session_id
group by
    session_info.date,
    session_info.country,
    session_info.continent,
    session_info.channel,
    session_info.device,
    session_info.test,
    session_info.test_group
),

events as(
  SELECT
    session_info.date,
    session_info.country,
    session_info.continent,
    session_info.channel,
    session_info.device,
    session_info.test,
    session_info.test_group,
    sp.event_name,
    count(sp.ga_session_id) as event_cnt
from `DA.event_params` sp
join session_info
on sp.ga_session_id = session_info.ga_session_id
group by
    session_info.date,
    session_info.country,
    session_info.continent,
    session_info.channel,
    session_info.device,
    session_info.test,
    session_info.test_group,
    sp.event_name
),
session as (
  SELECT
    session_info.date,
    session_info.country,
    session_info.continent,
    session_info.channel,
    session_info.device,
    session_info.test,
    session_info.test_group,
    COUNT(distinct session_info.ga_session_id) as session_cnt
  from session_info
  group by
    session_info.date,
    session_info.country,
    session_info.continent,
    session_info.channel,
    session_info.device,
    session_info.test,
    session_info.test_group
),

account as (
SELECT
    session_info.date,
    session_info.country,
    session_info.continent,
    session_info.channel,
    session_info.device,
    session_info.test,
    session_info.test_group,
    COUNT(distinct acs.ga_session_id) as new_acc_cnt
FROM `DA.account_session` acs
join session_info
on acs.ga_session_id = session_info.ga_session_id
group by
    session_info.date,
    session_info.country,
    session_info.continent,
    session_info.channel,
    session_info.device,
    session_info.test,
    session_info.test_group
)

SELECT
    session_with_orders.date,
    session_with_orders.country,
    session_with_orders.continent,
    session_with_orders.channel,
    session_with_orders.device,
    session_with_orders.test,
    session_with_orders.test_group,
    'session_with_orders' as event_name,
    session_with_orders.session_with_orders as value
FROM session_with_orders

union all
SELECT
    events.date,
    events.country,
    events.continent,
    events.channel,
    events.device,
    events.test,
    events.test_group,
    events.event_name,
    event_cnt as value
FROM events

union all
SELECT
    session.date,
    session.country,
    session.continent,
    session.channel,
    session.device,
    session.test,
    session.test_group,
    'session' as event_name,
    session_cnt as value
FROM session

union all
SELECT
    account.date,
    account.country,
    account.continent,
    account.channel,
    account.device,
    account.test,
    account.test_group,
    'new account' as event_name,
    new_acc_cnt as value
FROM account
"""

In [ ]:
# Виконання запит та перетворення результатів у pandas DataFrame
df = client.query(query).to_dataframe()

In [ ]:
# Виведення перших 5 рядків DataFrame
print(df.head())

         date          country continent         channel   device  test  \
0  2020-11-01        Lithuania    Europe  Organic Search   mobile     2   
1  2020-11-01      El Salvador  Americas   Social Search  desktop     2   
2  2020-11-01         Slovakia    Europe     Paid Search   mobile     2   
3  2020-11-01        Lithuania    Europe     Paid Search  desktop     2   
4  2020-11-02  North Macedonia    Europe          Direct  desktop     2   

   test_group   event_name  value  
0           2  new account      1  
1           1  new account      1  
2           2  new account      1  
3           2  new account      1  
4           1  new account      1  


In [ ]:
# Перевірка типів даних
print("\nТипи даних у датасеті:")
print(df.dtypes)

# Загальна кількість колонок
total_columns = df.shape[1]
print("Загальна кількість колонок:", total_columns)

# Визначення числових колонок
numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
print("Кількість числових колонок:", len(numeric_cols))
print("Числові колонки:", numeric_cols)

# Визначення категоріальних колонок
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
print("Кількість категоріальних колонок:", len(categorical_cols))
print("Категоріальні колонки:", categorical_cols)

# Визначення datetime колонок
datetime_cols = df.select_dtypes(include=['datetime64[ns]']).columns.tolist()
print("Кількість колонок типу datetime:", len(datetime_cols))


Типи даних у датасеті:
date          dbdate
country       object
continent     object
channel       object
device        object
test           Int64
test_group     Int64
event_name    object
value          Int64
dtype: object
Загальна кількість колонок: 9
Кількість числових колонок: 3
Числові колонки: ['test', 'test_group', 'value']
Кількість категоріальних колонок: 5
Категоріальні колонки: ['country', 'continent', 'channel', 'device', 'event_name']
Кількість колонок типу datetime: 0


In [ ]:
# Кількість унікальних сесій (за кількістю унікальних комбінацій дати та тестової групи, пристрою тощо)
if 'event_name' in df.columns and 'value' in df.columns:
    total_sessions = df.loc[df['event_name'] == 'session', 'value'].sum()
    print("Кількість унікальних сесій:", total_sessions)

# Визначення періоду часу, який охоплює датафрейм
if 'date' in df.columns:
    start_date = df['date'].min()
    end_date = df['date'].max()
    print(f"Період часу (date): від {start_date} до {end_date}")

Кількість унікальних сесій: 542142
Період часу (date): від 2020-11-01 до 2021-01-27


In [ ]:
# Метрики для аналізу
metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new account']

# Всі тести з групами (1-2 в кожному)
tests_and_groups = [(1, 1, 2), (2, 1, 2), (3, 1, 2), (4, 1, 2)]

# Порожній список для збору результатів
results = []

for test_num, control_group, event_group in tests_and_groups:
    for metric in metrics:
        control_event = df[(df['event_name'] == metric) & (df['test'] == test_num) & (df['test_group'] == control_group)]
        event_event = df[(df['event_name'] == metric) & (df['test'] == test_num) & (df['test_group'] == event_group)]

        control_session = df[(df['event_name'] == 'session') & (df['test'] == test_num) & (df['test_group'] == control_group)]
        event_session = df[(df['event_name'] == 'session') & (df['test'] == test_num) & (df['test_group'] == event_group)]

        numerator_control = control_event['value'].sum()
        denominator_control = control_session['value'].sum()
        numerator_event = event_event['value'].sum()
        denominator_event = event_session['value'].sum()

        conversion_rate_control = numerator_control / denominator_control if denominator_control > 0 else np.nan
        conversion_rate_event = numerator_event / denominator_event if denominator_event > 0 else np.nan

        metric_change = (conversion_rate_event / conversion_rate_control - 1) * 100 if conversion_rate_control > 0 else np.nan

        if denominator_control > 0 and denominator_event > 0:
            counts = np.array([numerator_event, numerator_control])
            nobs = np.array([denominator_event, denominator_control])
            z_stat, p_value = proportions_ztest(counts, nobs)
        else:
            z_stat, p_value = np.nan, np.nan

        results.append({
            'test_number': test_num,
            'metric': metric,
            'numerator_event': numerator_event,
            'denominator_event': denominator_event,
            'numerator_control': numerator_control,
            'denominator_control': denominator_control,
            'conversion_rate_event': conversion_rate_event,
            'conversion_rate_control': conversion_rate_control,
            'metric_change': metric_change,
            'z_stat': z_stat,
            'p_value': p_value,
            'significant': p_value < 0.05 if not np.isnan(p_value) else False
        })

# Створення фінального датафрейму
final_df = pd.DataFrame(results)

# Вивід перших рядків результату
final_df.head(20)

,test_number,metric,numerator_event,denominator_event,numerator_control,denominator_control,conversion_rate_event,conversion_rate_control,metric_change,z_stat,p_value,significant
0,1,add_payment_info,2229,45193,1988,45362,0.049322,0.043825,12.542021,3.924884,0.000087,True
1,1,add_shipping_info,3221,45193,3034,45362,0.071272,0.066884,6.560481,2.603571,0.009226,True
2,1,begin_checkout,4021,45193,3784,45362,0.088974,0.083418,6.660587,2.978783,0.002894,True
3,1,new account,3681,45193,3823,45362,0.081451,0.084278,-3.354299,-1.542883,0.122859,False
4,2,add_payment_info,2409,50244,2344,50637,0.047946,0.046290,3.576911,1.240994,0.214608,False
5,2,add_shipping_info,3510,50244,3480,50637,0.069859,0.068724,1.650995,0.709557,0.477979,False
6,2,begin_checkout,4313,50244,4262,50637,0.085841,0.084168,1.988164,0.952898,0.340642,False
7,2,new account,4184,50244,4165,50637,0.083274,0.082252,1.241934,0.588793,0.556000,False
8,3,add_payment_info,3697,70439,3623,70047,0.052485,0.051722,1.474630,0.643172,0.520112,False
9,3,add_shipping_info,5188,70439,5298,70047,0.073652,0.075635,-2.621211,-1.413727,0.157442,False


In [ ]:
# Експорт розрахунків у xlsx файл
final_df.to_excel("ab_test_final_results.xlsx", index=False)
files.download("ab_test_final_results.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🔽🔽🔽Візуалізація в Табло🔽🔽🔽

https://public.tableau.com/views/ab_17461856175450/abtest?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link